https://programmer.help/blogs/fifty-sql-exercise-questions-and-answers-and-detailed-analysis.html

In [ ]:
# Hier ist nur Code zum Initialisieren der Umgebeung, bitte gehen Sie weiter, es gibt nichts zu sehen.

# Keine langen Fehlermeldungen
import sys
ipython = get_ipython()

def exception_handler(exception_type, exception, traceback):
    print("%s: %s" % (exception_type.__name__, exception), file=sys.stderr)

ipython._showtraceback = exception_handler

# Lade die Erweiterung, damit wir SQL Befehle nutzen können
%load_ext sql

# Verbinde Dich zu einer in - Memory Datenbank
%sql sqlite:///uni.db
%sql PRAGMA foreign_keys = ON

In [ ]:
!echo .schema | sqlite3 uni.db

![ER Diagramm](https://nextcloud.th-deg.de/s/Kc7m486Z3KbZyB4/download?path=%2F&files=hochschule-er.png)

# Aufgabe 1

Welche Studenten haben in der Prüfung der Vorlesung "Logik" mehr Punkte als in der Vorlesung "Ethik"

# Aufgabe 2

Geben Sie Matrikelnummer, name und durchschnittsnote von Studenten aus, deren Durschnittsnote besser als 3 ist.

# Aufgabe 3

Welche Studenten haben in welchem Fach mit welcher Note bereits an mindestens einer Prüfung teilgenommen

# Aufgabe 4

Welche Studenten haben schon Prüfungen gesammelt?

# Aufgabe 5

Geben Sie alle Informationen zu jedem Studierenden aus. Darüber hinaus geben Sie noch die Anzahl der aktuell gehörten Kurse, die Anzahl der bereits gehörten Kurse und die durschnittsnote aus. 

# Aufgabe 6

Wie viele Professoren heißen was mit "krates" am Ende?

# Aufgabe 7
Welche Studenten wurden von Sokrates geprüft?

# Aufgabe 8

Welche Studenten hören weder eine Vorlesung, noch haben eine Prüfung abgelegt?

# Aufgabe 10

Welche Stundenten haben noch nie einen Kurs von Sokratest belegt?

# Aufgabe 11

Suche Studenten (id, name, durchschnittliche Punktzahl), die zwei oder mehr Kurse nicht bestanden haben

oder:

# Aufgabe 12

Welche Studenten sind in Ethik durchgefallen? Sortieren Sie nach Matirkelnummer absteigend.

# Aufgabe 13

Zeige für jeden Studenten die Noten aller Kurse im Vergleich zur durchschnittlich erreichten Punktzahl in diesem Kurs

# Aufgabe 14

Beste, Schlechteste und Durchschnittsnote pro Kurs

# Aufgabe 15

Erstellen Sie ein Ranking der besten Studenten pro Kurs. Geben Sie den Namen des Kurses, den Namen des Studenten, dessen Note und dessen Ranking an.

# Aufgabe 16

Finden Sie die beiden besten Studenten pro Vorlesung